# rnn

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2024/02/image-80.png)

_For those interested in how the forward and backward pass works._

RNNs are a type of neural net that operate on sequences. They use a hidden state vector and an input vector to predict an output vector (ie. probabilities over tokens) and the next hidden state. The next hidden state is used to predict the next token, so on and so forth. The recurrence relationship, where the next hidden state depends on the previous hidden state gives them the name **Recurrent Neural Nets**.

## Forward Pass

RNNs have a simple API. They take in a hidden state vector and an input vector to produce an output vector and the next hidden state.

**Weights and Bias Matrices**
$$W_{xh}\in ({n \times m}), \ W_{hh} \in ({n \times n}), b_h \in ({1 \times n}) $$
$$W_{hy} \in ({l \times n}), b_y \in ({1 \times l})$$

**Input Vector**
$$x_t \in ({1 \times m})$$

**Hidden State Vector**
$$h_t \in ({1 \times n})$$

**Foward Pass**

We have all the ingredients for the forward pass! Our choice of activation is `tanh` and `softmax`. Tanh squeezes the activations between -1 and 1 and softmax gives us output probabilities.

$$
z_t^h = x_tW_{xh}^{T} + h_{t-1}W_{hh}^{T} + b_h
$$

$$
h_t = \tanh(a_t)
$$

$$
z_t^y = h_tW_{hy}^{T} + b_y
$$

$$
\hat{y}_t = \text{softmax}(b_t)
$$

## Backward Pass

After the forward pass, we'll compute the loss and gradients of the weight and bias matrices. Then do gradient descent.

### Loss

We use cross entropy loss between the predicted token probability and the target token across all time steps.

$$
L = \sum_{t=1}^{T}L_t(\hat{y}_t, y_t) = \sum_{t=1}^{T}{\sum_{i = 1}^{C}-y_t^i
\log(\hat{y}_t^i)}
$$

### Backpropagation Through Time

Source: https://phillipi.github.io/6.882/2020/notes/6.036_notes.pdf

The hardest part is keeping track of matrix shapes. Do multiply the shapes of the partials to check if the shapes make sense. Don't memorize. You can derive everything from first principles just by following the RNN section of the textbook above. Phillip Isola does an amazing job at explaining how to implement BPTT and where the gradients come from. 

Hopefully you'll come to the same result! My derivation is slightly different because I use row vectors instead of column vectors $x_t \in (1 \times m)$.

Just peel back each layer and apply the chain rule.


#### Gradient of Loss w.r.t $z_t^y$

https://cs231n.github.io/neural-networks-case-study/#grad

$$
\frac{\partial L_t}{\partial z_t^y} = \hat{y_t} - y_t
$$

$$(1 \times l) = (1 \times l) - (1 \times l)$$

#### Gradient of Tanh
$$\frac{\partial \tanh(u)}{\partial u} = 1 - \tanh(u)^2$$

#### Gradient of 1st Layer

Gradient of the $W_{hy}$

$$
\frac{\partial L}{\partial W_{hy}} = \sum_{t=1}^T \frac{\partial L_t}{\partial z_t^y}\frac{\partial z_t^y}{\partial W_{hy}} = \sum_{t=1}^T (\frac{\partial L_t}{\partial z_t^y})^T h_t
$$

$$
(l \times n) = (l \times 1) \times (1 \times n)
$$

Gradient of the $b_y$

$$
\frac{\partial L}{\partial b_y} = \sum_{t=1}^T \frac{\partial L_t}{\partial z_t^y}\frac{\partial z_t^y}{\partial b_y} = \frac{\partial L_t}{\partial z_t^y}
$$

$$
(1 \times l) = (1 \times l)
$$


#### Gradients of 2nd Layer

This is the most tricky layer. Let's define some terms which will be useful.

#### Definitions

We'll be taking gradients of the future loss with respect to a hidden state. The future loss is defined as follows. Note the recurrence in the definition (we can write $F_{t-1}$ in terms of $F_t$).

$$F_t = \sum_{u = t + 1}^TL_u$$

$$F_{t - 1} = L_t + \sum_{u = t + 1}^TL_u = L_t + F_t$$

#### Gradient of Future Loss w.r.t hidden state

$$
\delta^{h_{t-1}} = \frac{\partial F_{t-1}}{\partial h_{t-1}} 
= \frac{\partial}{\partial h_{t-1}} {\sum_{u = t}^TL_u} 
= \frac{\partial}{\partial h_t} {\sum_{u = t}^TL_u} \frac{\partial h_t}{\partial h_{t-1}}
= (\frac{\partial L_t}{\partial h_t} + \frac{\partial}{\partial h_t}\sum_{u = t + 1}^TL_u) \frac{\partial h_t}{\partial h_{t-1}}
= (\frac{\partial L_t}{\partial h_t} + \delta^{h_t}) \frac{\partial h_t}{\partial h_{t-1}} = \frac{\partial F_{t-1}}{\partial h_t} \frac{\partial h_t}{\partial h_{t-1}}
$$

$$
(1 \times n)
$$

Note that $\frac{\partial L_t}{\partial h_t} + \delta^{h_t} = \frac{\partial F_{t-1}}{\partial h_t}$.


Also note that $\frac{\partial F_T}{\partial h_T} = 0 \in (1 \times n)$ because there $L_{T+1} ...$ do not exist. $L_T$ is the loss at the final timestep, so the final hidden state $h_T$ will have no effect on future losses, hence the zero gradient. 

#### Gradient of Loss w.r.t hidden state
$$
\frac{\partial L_t}{\partial h_t} = \frac{\partial L_t}{\partial z_t^y}\frac{\partial z_t^y}{\partial h_t} = \frac{\partial L_t}{\partial z_t^y} W_{hy}
$$

$$
(1 \times n) = (1 \times l) \times (l \times n)
$$

#### Gradient of hidden state w.r.t to its input before activation $z_t^h$

$$\frac{\partial h_t}{\partial z_t^h} = 1 - h_t^2$$

$$(1 \times n)$$

Really this is a $(n \times n)$ diagonal matix but b/c $\frac{\partial h_t^i}{\partial z_t^j} = 0$ when $i \neq j$, I decided to grab diagonal and stuff it into a $(1 \times n)$ row vector. The reason being activation are applied element-wise.


#### Gradient of $F_{t-1}$ w.r.t hidden state

Everything is going to come together nicely now. This is just the sum of 2 gradients we defined above.

$$
\frac{\partial F_{t-1}}{\partial h_t} = \frac{\partial L_t}{\partial h_t} + \frac{\partial}{\partial h_t} \sum_{u = t + 1}^TL_u = \frac{\partial L_t}{\partial h_t} + \delta^{h_t}
$$

$$
(1 \times n) = (1 \times n) + (1 \times n)
$$


#### Gradient of $F_{t-1}$ w.r.t $z_t^h$

$$
\frac{\partial F_{t-1}}{\partial z_t^h} = \frac{\partial F_{t-1}}{\partial h_t} \frac{\partial h_t}{\partial z_t^h}
$$

$$
(1 \times n) = (1 \times n) \times (1 \times n)
$$


#### Gradient of hidden state weight matrices

Let's calculate them now!

$$
\frac{\partial L}{\partial W_{xh}} = \sum_{t=1}^{T} \frac{\partial F_{t-1}}{\partial h_t} \frac{\partial h_t}{\partial z_t^h} \frac{\partial z_t^h}{\partial W_{xh}} = \sum_{t=1}^T (\frac{\partial F_{t-1}}{\partial z_t^h})^T x_t
$$

$$
(n \times m) = (n \times 1) \times (1 \times m)
$$

$$
\frac{\partial L}{\partial W_{hh}} = \sum_{t=1}^{T} \frac{\partial F_{t-1}}{\partial h_t} \frac{\partial h_t}{\partial z_t^h} \frac{\partial z_t^h}{\partial W_{xh}} = \sum_{t=1}^T (\frac{\partial F_{t-1}}{\partial z_t^h})^T h_{t-1}
$$


$$
(n \times n) = (n \times 1) \times (1 \times n)
$$

#### Gradient of hidden state bias vector
$$
\frac{\partial L}{\partial b_{h}} = \sum_{t=1}^{T} \frac{\partial F_{t-1}}{\partial h_t} \frac{\partial h_t}{\partial z_t^h} \frac{\partial z_t^h}{\partial b_h} = \sum_{t=1}^T \frac{\partial F_{t-1}}{\partial z_t^h}
$$

$$
(1 \times n) = (1 \times n)
$$

#### Computing $\delta^h_{t-1}$
At timestep $t$ to compute $\frac{\partial F_{t-1}}{\partial h_t}$, we need $\delta^h_t$. For timestep $t-1$, we will need to compute $\delta^h_{t-1}$. Let's revisit the definition of this gradient.

$$
\delta^{h_{t-1}} = \frac{\partial F_{t-1}}{\partial h_t} \frac{\partial h_t}{\partial h_{t-1}}
$$

Let's apply the chain rule to $\frac{\partial h_t}{\partial h_{t-1}}$. 

$$
\delta^{h_{t-1}} = \frac{\partial F_{t-1}}{\partial h_t} \frac{\partial h_t}{\partial z_t^h}\frac{\partial z_t^h}{\partial h_{t-1}}
$$

Simplify.

$$
\delta^{h_{t-1}} = \frac{\partial F_{t-1}}{\partial z_t^h} \frac{\partial z_t^h}{\partial h_{t-1}} = \frac{\partial F_{t-1}}{\partial z_t^h} W_{hh}
$$

$$(1 \times n) = (1 \times n) \times (n \times n)$$

We now have everything we need to implement backprop. In `rnn.py` we will translate all of this to code.

### Gradient Descent

We'll implement a version of gradient descent called Adagrad. One of the common problems training RNNs are the exploding vanishing gradients. Adagrad *adapts* our learning rate so that we take smaller steps when the gradients are big and bigger steps when gradients are small. This improves our training stability significantly. In fact, using vanilla stochastic gradient descent, training does not converge.

$$
g_t = \nabla_{\theta} L(\theta_t)
$$
$$
G_t = G_{t-1} + g_t^2
$$
$$
\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{G_t + \epsilon}} g_t
$$

We implement this in `train.py`.

## Resources
Would not have made it without these. Read them.

- https://phillipi.github.io/6.882/2020/notes/6.036_notes.pdf
- https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks
- https://karpathy.github.io/2015/05/21/rnn-effectiveness/
- https://gist.github.com/karpathy/d4dee566867f8291f086
- https://cs231n.github.io/neural-networks-case-study/#grad
- https://explained.ai/matrix-calculus/
- https://www.youtube.com/watch?v=0XdPIqi0qpg


In [5]:
import numpy as np
from tqdm import tqdm
import re
import random

## Build RNN

In [2]:
def softmax(z, t: float = 1.0):
    """Softmax w/ temperature t"""
    z_exp = np.exp(z / t)
    return z_exp / np.sum(z_exp)


class RNN:
    """Vanilla character-level RNN in numpy"""

    def __init__(self, hidden_size: int, vocab: list[str]):
        """
        Initialize the weight and bias matrices

        Params:
            - hidden_size: hidden state dim
            - vocab: list of unique chars
        """
        # params + tokenizer
        self.hidden_size = hidden_size
        self.start_token = "<S>"
        self.eos_token = "<EOS>"
        self.start_token_idx = 0
        self.eos_token_idx = 1
        self.vocab = [self.start_token, self.eos_token] + vocab
        self.vocab_size = len(self.vocab)
        self.char_to_idx = {char: i for i, char in enumerate(self.vocab)}
        self.idx_to_char = {i: char for i, char in enumerate(self.vocab)}

        # weights
        self.Wxh = np.random.randn(hidden_size, self.vocab_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Why = np.random.randn(self.vocab_size, hidden_size) * 0.01

        # biases
        self.bh = np.zeros((1, hidden_size))
        self.by = np.zeros((1, self.vocab_size))

        self.weights = (self.Wxh, self.Whh, self.Why, self.bh, self.by)
        self.num_params = sum(w.size for w in self.weights)

    def encode(self, chars: str):
        """Turns a string of chars into idxes"""
        ids = (
            [self.start_token_idx]
            + [self.char_to_idx[char] for char in chars]
            + [self.eos_token_idx]
        )
        return ids

    def decode(self, idxes: list[int]):
        """Turns a list of idxes into chars"""
        chars = [
            self.idx_to_char[idx]
            for idx in idxes
            if idx != self.eos_token_idx and idx != self.start_token_idx
        ]
        return "".join(chars)

    def __call__(self, x, h, t=1.0) -> np.ndarray:
        """RNN forward pass at softmax temperature t"""
        assert x.shape == (1, self.vocab_size) and h.shape == (1, self.hidden_size)
        zh = x @ self.Wxh.T + h @ self.Whh.T + self.bh
        hnext = np.tanh(zh)
        zy = hnext @ self.Why.T + self.by
        y = softmax(zy, t)

        return y, hnext

    def sample(self, t: float = 1.0, n: int = 1000):
        """Generates unconditional samples with temperature t, with max tokens n"""
        sample = []
        for i in self.sample_progressive(t, n):
            sample.append(i)

        return sample

    def sample_progressive(self, t: float = 1.0, n: int = 1000):
        """Generate one char at a time, starting at temperature t with max tokens n"""
        x = np.zeros((1, self.vocab_size))
        x[0, self.start_token_idx] = 1  # create one hot encoding for char
        h = np.zeros((1, self.hidden_size))  # initialize hidden state to all 0s

        for i in range(n):
            probs, h = self(x, h, t)  # sample next token probs
            idx = np.random.choice(self.vocab_size, p=probs.ravel())

            x = np.zeros((1, self.vocab_size))
            x[0, idx] = 1  # one hot encoding for sampled token

            yield idx

            if idx == self.eos_token_idx:  # stop sampling
                break

    def loss(self, inputs: list[int], targets: list[int], hprev=None):
        """
        Computes loss between input and target chars idxes and returns
        the loss, gradients (dWxh, dWhh, dWhy, dbh, dby), and final hidden state
        """
        assert len(inputs) == len(targets)
        xs, hs, ps = {}, {}, {}  # keep track of x, hidden states, and output probs
        hs[-1] = (
            hprev if hprev is not None else np.zeros((1, self.hidden_size))
        )  # store initial hidden state
        loss = 0

        # forward pass
        for t in range(len(inputs)):
            x = np.zeros((1, self.vocab_size))
            x[0, inputs[t]] = 1  # one hot encoding
            p, h = self(x, hs[t - 1])  # rnn

            xs[t] = x  # store x, hidden state, probs (we'll need them for backprop)
            hs[t] = h
            ps[t] = p
            loss += -np.log(ps[t][0, targets[t]])

        # gradient of loss w.r.t weights and biases
        dWxh, dWhh, dWhy = (
            np.zeros_like(self.Wxh),
            np.zeros_like(self.Whh),
            np.zeros_like(self.Why),
        )
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)

        # gradient of F_t (future loss L_t+1 ... L_T) w.r.t. h_t
        dFdh = np.zeros((1, self.hidden_size))

        # backprop thr. time
        for t in reversed(range(len(inputs))):
            dzy = np.copy(ps[t])  # loss at t w.r.t zy
            dzy[:, targets[t]] -= 1

            # 2nd layer
            dWhy += dzy.T @ hs[t]
            dby += dzy

            # intermediate gradients
            dLdh = dzy @ self.Why  # gradient of loss at L_t w.r.t hidden state h_t
            dhdzh = 1 - hs[t] ** 2  # gradient thr. tanh activation
            dFprevdh = dLdh + dFdh  # gradient of F_{t-1} w.r.t h_t
            dFprevdzh = dFprevdh * dhdzh  # gradient of F_{t-1} w.r.t z_h

            # 1st layer
            dWxh += dFprevdzh.T @ xs[t]
            dWhh += dFprevdzh.T @ hs[t - 1]
            dbh += dFprevdzh

            dFdh = dFprevdzh @ self.Whh  # update dFdh for next timestep t-1

        gradients = (dWxh, dWhh, dWhy, dbh, dby)  # collect gradients
        hnext = hs[len(inputs) - 1]  # final hidden state

        return loss, gradients, hnext

## Util Functions

In [3]:
def read_file(path: str):
    with open(path, "r") as f:
        return f.read()


def build_vocab(data: str) -> str:
    chars = set()
    vocab = []
    for char in data:
        if char not in chars:
            vocab.append(char)
            chars.add(char)

    return vocab

def clip_gradients(gradients):
    for gradient in gradients:
        np.clip(gradient, -5, 5, out=gradient)  # clip gradient in-place

## Load Dataset

In [ ]:
data = read_file("./weights/stevejobs.txt")  # read txt file
dataset = re.split(r"\n\s*\n", data)  # split data into paragraphs
vocab = build_vocab(data)  # build vocab of unique chars

## Train Loop

In [ ]:
def train_loop(
    rnn: RNN,
    dataset: list[str],
    iters: int,
    lr: float,
    seq_length: int,
    val_steps: int,
    val_t: float,
):
    print("[INFO] Training...")
    print(f"[INFO] data_size = {len(''.join(dataset))}")
    print(f"[INFO] num_params = {rnn.num_params}")
    print(f"[INFO] vocab_size = {rnn.vocab_size}")
    print(f"[INFO] hidden_size = {rnn.hidden_size}")
    print(f"[INFO] lr = {lr}")
    print(f"[INFO] iters = {iters}")
    print(f"[INFO] seq_length = {seq_length}")

    # memory variables for Adagrad
    mWxh, mWhh, mWhy = (
        np.zeros_like(rnn.Wxh),
        np.zeros_like(rnn.Whh),
        np.zeros_like(rnn.Why),
    )
    mbh, mby = np.zeros_like(rnn.bh), np.zeros_like(rnn.by)

    n = 0
    pbar = tqdm(total=iters, position=0)
    smooth_loss = -np.log(1.0 / rnn.vocab_size) * seq_length  # loss at iteration 0
    smooth_perplexity = 2**smooth_loss

    while True:
        batches = random.sample(dataset, len(dataset))
        for minibatch in batches:
            hprev = np.zeros((1, rnn.hidden_size))  # reset RNN memory
            minibatch_idxs = rnn.encode(minibatch)

            for p in range(0, len(minibatch), seq_length):
                batch = minibatch_idxs[p : p + seq_length + 1]
                inputs, targets = batch[:-1], batch[1:]

                # compute loss and clip gradients
                loss, gradients, hprev = rnn.loss(inputs, targets, hprev)
                smooth_loss = smooth_loss * 0.999 + loss * 0.001
                smooth_perplexity = 2**smooth_loss
                clip_gradients(gradients)

                # sample
                if (n + 1) % val_steps == 0 or n == 0 or n == iters - 1:
                    tqdm.write(
                        f"iter: {n + 1}, loss: {smooth_loss}, perplexity: {smooth_perplexity}"
                    )
                    idxes = rnn.sample(val_t)
                    txt = rnn.decode(idxes)
                    tqdm.write(f"----\n{txt}\n----")
                    tqdm.write("")

                # adagrad gradient descent
                for param, dparam, mem in zip(
                    rnn.weights,
                    gradients,
                    [mWxh, mWhh, mWhy, mbh, mby],
                ):
                    mem += dparam**2
                    param += -lr * dparam / np.sqrt(mem + 1e-8)  # adagrad update

                n += 1
                pbar.update(1)

                if n >= iters:
                    break
            if n >= iters:
                break
        if n >= iters:
            break

    print("[INFO] Training complete!")

## Hyperparams

In [ ]:
hidden_size = 128
iters = 100000
lr = 1e-1
seq_length = 25
val_steps = 1000
val_t = 0.5

## Training

In [ ]:
rnn = RNN(hidden_size=hidden_size, vocab=vocab)

train_loop(
    rnn=rnn,
    dataset=dataset,
    iters=iters,
    lr=lr,
    seq_length=seq_length,
    val_steps=val_steps,
    val_t=val_t,
)

## Inference

In [6]:
t = 0.5
sample = rnn.sample(t=t)
txt = rnn.decode(sample)
print(txt)

NameError: name 'rnn' is not defined